# Comprehensive Exploratory Data Analysis (EDA)
# Telecom Customer Churn Prediction

## Table of Contents
1. [Introduction and Business Understanding](#introduction)
2. [Data Import and Initial Exploration](#data-import)
3. [Data Cleaning and Preprocessing](#data-cleaning)
4. [Univariate Analysis](#univariate-analysis)
5. [Bivariate Analysis](#bivariate-analysis)
6. [Multivariate Analysis](#multivariate-analysis)
7. [Statistical Analysis](#statistical-analysis)
8. [Customer Segmentation Analysis](#segmentation)
9. [Churn Pattern Analysis](#churn-patterns)
10. [Key Insights and Business Recommendations](#insights)
11. [Conclusion](#conclusion)

---


## 1. Introduction and Business Understanding {#introduction}

### Problem Statement
Customer churn is a critical business metric for telecom companies. The cost of acquiring new customers is typically 5-25 times higher than retaining existing ones. This analysis aims to:

- Understand the factors that contribute to customer churn
- Identify patterns and trends in customer behavior
- Provide actionable insights for customer retention strategies
- Analyze customer segments and their churn propensity

### Dataset Overview
This dataset contains information about telecom customers including:
- **Demographics**: Gender, Age (Senior Citizen), Partner, Dependents
- **Services**: Phone, Internet, Online Security, Backup, etc.
- **Account**: Contract type, Payment method, Billing preferences
- **Usage**: Tenure, Monthly charges, Total charges
- **Target**: Churn (Yes/No)


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from scipy import stats
from scipy.stats import chi2_contingency
import plotly.figure_factory as ff

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")

## 2. Data Import and Initial Exploration {#data-import}

In [ ]:
# Load the dataset
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

print("Dataset loaded successfully!")
print(f"Dataset shape: {df.shape}")
print(f"Number of customers: {df.shape[0]:,}")
print(f"Number of features: {df.shape[1]}")

In [ ]:
# Display first few rows
print("First 5 rows of the dataset:")
df.head()

In [ ]:
# Dataset information
print("Dataset Information:")
print("="*50)
df.info()

In [ ]:
# Check for missing values
print("Missing Values Analysis:")
print("="*30)
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': missing_values,
    'Missing_Percentage': missing_percentage
})
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_df) > 0:
    print(missing_df)
else:
    print("No missing values found!")

In [ ]:
# Basic descriptive statistics
print("Descriptive Statistics for Numerical Features:")
print("="*50)
df.describe()

In [ ]:
# Check unique values for each column
print("Unique Values Analysis:")
print("="*30)
for column in df.columns:
    unique_count = df[column].nunique()
    print(f"{column}: {unique_count} unique values")
    if unique_count <= 10:
        print(f"   Values: {df[column].unique()}")
    print()

## 3. Data Cleaning and Preprocessing {#data-cleaning}

In [ ]:
# Create a copy for processing
df_clean = df.copy()

print("Original dataset shape:", df.shape)

# Check if TotalCharges is string and convert to numeric
print("\nTotalCharges data type:", df_clean['TotalCharges'].dtype)
print("Sample TotalCharges values:")
print(df_clean['TotalCharges'].head(10))

In [ ]:
# Convert TotalCharges to numeric (handle empty strings)
df_clean['TotalCharges'] = pd.to_numeric(df_clean['TotalCharges'], errors='coerce')

# Check for any missing values after conversion
total_charges_missing = df_clean['TotalCharges'].isnull().sum()
print(f"Missing values in TotalCharges after conversion: {total_charges_missing}")

if total_charges_missing > 0:
    # Display rows with missing TotalCharges
    missing_rows = df_clean[df_clean['TotalCharges'].isnull()]
    print(f"\nRows with missing TotalCharges: {len(missing_rows)}")
    print("Sample missing rows:")
    print(missing_rows[['customerID', 'tenure', 'MonthlyCharges', 'TotalCharges']].head())
    
    # For customers with tenure 0 or very low, TotalCharges might be 0 or equal to MonthlyCharges
    # Fill missing TotalCharges with MonthlyCharges for new customers
    df_clean.loc[df_clean['TotalCharges'].isnull(), 'TotalCharges'] = df_clean.loc[df_clean['TotalCharges'].isnull(), 'MonthlyCharges']
    
    print(f"\nAfter filling: {df_clean['TotalCharges'].isnull().sum()} missing values")

In [ ]:
# Create categorical and numerical column lists
categorical_columns = []
numerical_columns = []
binary_columns = []

for column in df_clean.columns:
    if column == 'customerID':
        continue
    elif df_clean[column].dtype in ['object']:
        if df_clean[column].nunique() == 2:
            binary_columns.append(column)
        else:
            categorical_columns.append(column)
    else:
        numerical_columns.append(column)

print(f"Binary columns ({len(binary_columns)}): {binary_columns}")
print(f"Categorical columns ({len(categorical_columns)}): {categorical_columns}")
print(f"Numerical columns ({len(numerical_columns)}): {numerical_columns}")

In [ ]:
# Create derived features for better analysis
print("Creating derived features...")

# Monthly charges per year of tenure (customer value intensity)
df_clean['AvgChargesPerTenure'] = df_clean['TotalCharges'] / (df_clean['tenure'] + 1)  # +1 to avoid division by zero

# Customer lifetime value tier
df_clean['CLV_Tier'] = pd.cut(df_clean['TotalCharges'], 
                             bins=[0, 1000, 3000, 5000, float('inf')], 
                             labels=['Low', 'Medium', 'High', 'Premium'])

# Tenure groups
df_clean['TenureGroup'] = pd.cut(df_clean['tenure'], 
                               bins=[0, 12, 24, 48, 72], 
                               labels=['0-1 year', '1-2 years', '2-4 years', '4+ years'])

# Monthly charges tier
df_clean['MonthlyChargesTier'] = pd.cut(df_clean['MonthlyCharges'], 
                                      bins=[0, 35, 65, 95, float('inf')], 
                                      labels=['Low', 'Medium', 'High', 'Premium'])

# Senior citizen flag (more readable)
df_clean['SeniorCitizenFlag'] = df_clean['SeniorCitizen'].map({0: 'No', 1: 'Yes'})

# Count of additional services
service_columns = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
df_clean['AdditionalServicesCount'] = df_clean[service_columns].apply(lambda x: (x == 'Yes').sum(), axis=1)

# Has dependents or partner (family status)
df_clean['HasFamily'] = ((df_clean['Partner'] == 'Yes') | (df_clean['Dependents'] == 'Yes')).map({True: 'Yes', False: 'No'})

print("Derived features created successfully!")
print(f"New dataset shape: {df_clean.shape}")

## 4. Univariate Analysis {#univariate-analysis}

Let's explore each variable individually to understand the distribution and characteristics.

In [ ]:
# Target variable distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Count plot
churn_counts = df_clean['Churn'].value_counts()
axes[0].pie(churn_counts.values, labels=churn_counts.index, autopct='%1.1f%%', startangle=90)
axes[0].set_title('Customer Churn Distribution', fontsize=14, fontweight='bold')

# Bar plot
sns.countplot(data=df_clean, x='Churn', ax=axes[1])
axes[1].set_title('Churn Count', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Count')

# Add count labels on bars
for i, v in enumerate(churn_counts.values):
    axes[1].text(i, v + 50, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

churn_rate = (df_clean['Churn'] == 'Yes').mean()
print(f"\nOverall Churn Rate: {churn_rate:.2%}")
print(f"Total Customers: {len(df_clean):,}")
print(f"Churned Customers: {(df_clean['Churn'] == 'Yes').sum():,}")
print(f"Retained Customers: {(df_clean['Churn'] == 'No').sum():,}")

In [ ]:
# Numerical variables distribution
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

numerical_cols = ['tenure', 'MonthlyCharges', 'TotalCharges', 'AdditionalServicesCount']

for i, col in enumerate(numerical_cols):
    # Histogram with KDE
    sns.histplot(data=df_clean, x=col, kde=True, ax=axes[i])
    axes[i].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
    axes[i].axvline(df_clean[col].mean(), color='red', linestyle='--', alpha=0.7, label=f'Mean: {df_clean[col].mean():.2f}')
    axes[i].axvline(df_clean[col].median(), color='green', linestyle='--', alpha=0.7, label=f'Median: {df_clean[col].median():.2f}')
    axes[i].legend()

plt.tight_layout()
plt.show()

# Statistical summary
print("\nStatistical Summary of Numerical Variables:")
print("="*60)
print(df_clean[numerical_cols].describe())

In [ ]:
# Categorical variables distribution
cat_cols = ['gender', 'SeniorCitizenFlag', 'Partner', 'Dependents', 'PhoneService', 'InternetService']

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, col in enumerate(cat_cols):
    value_counts = df_clean[col].value_counts()
    
    # Create pie chart
    wedges, texts, autotexts = axes[i].pie(value_counts.values, labels=value_counts.index, 
                                          autopct='%1.1f%%', startangle=90)
    axes[i].set_title(f'{col} Distribution', fontsize=12, fontweight='bold')
    
    # Make percentage text bold
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_weight('bold')

plt.tight_layout()
plt.show()

In [ ]:
# Service-related categorical variables
service_cols = ['Contract', 'PaperlessBilling', 'PaymentMethod']

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, col in enumerate(service_cols):
    sns.countplot(data=df_clean, y=col, order=df_clean[col].value_counts().index, ax=axes[i])
    axes[i].set_title(f'{col} Distribution', fontsize=12, fontweight='bold')
    
    # Add count labels
    for j, v in enumerate(df_clean[col].value_counts().values):
        axes[i].text(v + 50, j, str(v), va='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Additional services analysis
additional_services = ['MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for i, service in enumerate(additional_services):
    if i < len(axes):
        value_counts = df_clean[service].value_counts()
        colors = sns.color_palette('husl', len(value_counts))
        
        wedges, texts, autotexts = axes[i].pie(value_counts.values, labels=value_counts.index, 
                                              autopct='%1.1f%%', startangle=90, colors=colors)
        axes[i].set_title(f'{service}', fontsize=11, fontweight='bold')
        
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_weight('bold')
            autotext.set_fontsize(9)

# Remove empty subplot
if len(additional_services) < len(axes):
    axes[-1].remove()

plt.tight_layout()
plt.show()

## 5. Bivariate Analysis {#bivariate-analysis}

Now let's explore relationships between variables and their impact on churn.

In [ ]:
# Churn rate by categorical variables
categorical_features = ['gender', 'SeniorCitizenFlag', 'Partner', 'Dependents', 'PhoneService', 'InternetService']

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.flatten()

for i, feature in enumerate(categorical_features):
    # Calculate churn rate by category
    churn_rate = df_clean.groupby(feature)['Churn'].apply(lambda x: (x=='Yes').mean()).reset_index()
    churn_rate.columns = [feature, 'ChurnRate']
    
    # Create grouped bar chart
    sns.barplot(data=df_clean, x=feature, y='Churn', estimator=lambda x: (x=='Yes').mean(), 
                ax=axes[i], palette='viridis')
    axes[i].set_title(f'Churn Rate by {feature}', fontsize=12, fontweight='bold')
    axes[i].set_ylabel('Churn Rate')
    axes[i].tick_params(axis='x', rotation=45)
    
    # Add percentage labels on bars
    for j, v in enumerate(churn_rate['ChurnRate']):
        axes[i].text(j, v + 0.01, f'{v:.1%}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Contract and Payment method analysis
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Contract type
contract_churn = df_clean.groupby('Contract')['Churn'].apply(lambda x: (x=='Yes').mean())
sns.barplot(x=contract_churn.index, y=contract_churn.values, ax=axes[0], palette='plasma')
axes[0].set_title('Churn Rate by Contract Type', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Churn Rate')
for i, v in enumerate(contract_churn.values):
    axes[0].text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold')

# Payment method
payment_churn = df_clean.groupby('PaymentMethod')['Churn'].apply(lambda x: (x=='Yes').mean())
sns.barplot(y=payment_churn.index, x=payment_churn.values, ax=axes[1], palette='plasma')
axes[1].set_title('Churn Rate by Payment Method', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Churn Rate')
for i, v in enumerate(payment_churn.values):
    axes[1].text(v + 0.02, i, f'{v:.1%}', va='center', fontweight='bold')

# Paperless billing
paperless_churn = df_clean.groupby('PaperlessBilling')['Churn'].apply(lambda x: (x=='Yes').mean())
sns.barplot(x=paperless_churn.index, y=paperless_churn.values, ax=axes[2], palette='plasma')
axes[2].set_title('Churn Rate by Paperless Billing', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Churn Rate')
for i, v in enumerate(paperless_churn.values):
    axes[2].text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Numerical variables vs Churn
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

numerical_features = ['tenure', 'MonthlyCharges', 'TotalCharges', 'AdditionalServicesCount']

for i, feature in enumerate(numerical_features):
    # Box plot
    sns.boxplot(data=df_clean, x='Churn', y=feature, ax=axes[i], palette='Set2')
    axes[i].set_title(f'{feature} by Churn Status', fontsize=12, fontweight='bold')
    
    # Add mean values
    means = df_clean.groupby('Churn')[feature].mean()
    for j, (churn_status, mean_val) in enumerate(means.items()):
        axes[i].text(j, mean_val, f'μ={mean_val:.1f}', ha='center', va='bottom', 
                    fontweight='bold', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

# Statistical summary by churn
print("\nNumerical Features Summary by Churn Status:")
print("="*60)
summary_stats = df_clean.groupby('Churn')[numerical_features].agg(['mean', 'median', 'std']).round(2)
print(summary_stats)

In [ ]:
# Correlation analysis
# First, encode categorical variables for correlation
df_encoded = df_clean.copy()

# Encode binary categorical variables
binary_encoding = {
    'gender': {'Male': 1, 'Female': 0},
    'Partner': {'Yes': 1, 'No': 0},
    'Dependents': {'Yes': 1, 'No': 0},
    'PhoneService': {'Yes': 1, 'No': 0},
    'PaperlessBilling': {'Yes': 1, 'No': 0},
    'Churn': {'Yes': 1, 'No': 0}
}

for col, mapping in binary_encoding.items():
    df_encoded[col] = df_encoded[col].map(mapping)

# One-hot encode other categorical variables
categorical_to_encode = ['InternetService', 'Contract', 'PaymentMethod']
df_encoded = pd.get_dummies(df_encoded, columns=categorical_to_encode, drop_first=True)

# Select numerical columns for correlation
numerical_cols_for_corr = ['SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 
                          'MonthlyCharges', 'TotalCharges', 'PaperlessBilling', 'Churn', 'AdditionalServicesCount']

# Calculate correlation matrix
correlation_matrix = df_encoded[numerical_cols_for_corr].corr()

# Create correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'label': 'Correlation Coefficient'})
plt.title('Correlation Matrix of Key Features', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Features most correlated with churn
churn_correlations = correlation_matrix['Churn'].abs().sort_values(ascending=False)
print("\nFeatures most correlated with Churn:")
print("="*40)
print(churn_correlations[1:])  # Exclude self-correlation

## 6. Multivariate Analysis {#multivariate-analysis}

Let's explore complex relationships between multiple variables.

In [ ]:
# Tenure vs Monthly Charges colored by Churn
plt.figure(figsize=(12, 8))
scatter = plt.scatter(df_clean['tenure'], df_clean['MonthlyCharges'], 
                     c=df_clean['Churn'].map({'Yes': 1, 'No': 0}), 
                     cmap='RdYlBu_r', alpha=0.6, s=50)
plt.colorbar(scatter, label='Churn (1=Yes, 0=No)')
plt.xlabel('Tenure (months)')
plt.ylabel('Monthly Charges ($)')
plt.title('Customer Tenure vs Monthly Charges (Colored by Churn Status)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Add trend lines
fig, ax = plt.subplots(figsize=(12, 8))
for churn_status in ['Yes', 'No']:
    subset = df_clean[df_clean['Churn'] == churn_status]
    ax.scatter(subset['tenure'], subset['MonthlyCharges'], 
              label=f'Churn: {churn_status}', alpha=0.6, s=30)
    
    # Add trend line
    z = np.polyfit(subset['tenure'], subset['MonthlyCharges'], 1)
    p = np.poly1d(z)
    ax.plot(subset['tenure'].sort_values(), p(subset['tenure'].sort_values()), 
           linestyle='--', linewidth=2)

ax.set_xlabel('Tenure (months)')
ax.set_ylabel('Monthly Charges ($)')
ax.set_title('Tenure vs Monthly Charges by Churn Status', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Churn rate by Internet Service and Contract type
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Internet Service and Contract
pivot_data = df_clean.pivot_table(values='Churn', index='InternetService', columns='Contract', 
                                 aggfunc=lambda x: (x=='Yes').mean())
sns.heatmap(pivot_data, annot=True, fmt='.2%', cmap='Reds', ax=axes[0])
axes[0].set_title('Churn Rate: Internet Service vs Contract Type', fontweight='bold')

# Payment Method and Contract
pivot_data2 = df_clean.pivot_table(values='Churn', index='PaymentMethod', columns='Contract', 
                                  aggfunc=lambda x: (x=='Yes').mean())
sns.heatmap(pivot_data2, annot=True, fmt='.2%', cmap='Reds', ax=axes[1])
axes[1].set_title('Churn Rate: Payment Method vs Contract Type', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Advanced segmentation analysis
# Create customer segments based on tenure and charges
def create_customer_segment(row):
    if row['tenure'] <= 12 and row['MonthlyCharges'] <= 50:
        return 'New Low-Value'
    elif row['tenure'] <= 12 and row['MonthlyCharges'] > 50:
        return 'New High-Value'
    elif row['tenure'] > 12 and row['tenure'] <= 36 and row['MonthlyCharges'] <= 50:
        return 'Established Low-Value'
    elif row['tenure'] > 12 and row['tenure'] <= 36 and row['MonthlyCharges'] > 50:
        return 'Established High-Value'
    elif row['tenure'] > 36 and row['MonthlyCharges'] <= 50:
        return 'Loyal Low-Value'
    else:
        return 'Loyal High-Value'

df_clean['CustomerSegment'] = df_clean.apply(create_customer_segment, axis=1)

# Analyze churn by customer segment
segment_analysis = df_clean.groupby('CustomerSegment').agg({
    'Churn': [lambda x: (x=='Yes').mean(), 'count'],
    'MonthlyCharges': 'mean',
    'TotalCharges': 'mean',
    'tenure': 'mean'
}).round(2)

segment_analysis.columns = ['ChurnRate', 'CustomerCount', 'AvgMonthlyCharges', 'AvgTotalCharges', 'AvgTenure']
segment_analysis = segment_analysis.sort_values('ChurnRate', ascending=False)

print("Customer Segment Analysis:")
print("="*60)
print(segment_analysis)

# Visualize segment analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Churn rate by segment
axes[0,0].bar(segment_analysis.index, segment_analysis['ChurnRate'])
axes[0,0].set_title('Churn Rate by Customer Segment', fontweight='bold')
axes[0,0].set_ylabel('Churn Rate')
axes[0,0].tick_params(axis='x', rotation=45)

# Customer count by segment
axes[0,1].bar(segment_analysis.index, segment_analysis['CustomerCount'])
axes[0,1].set_title('Customer Count by Segment', fontweight='bold')
axes[0,1].set_ylabel('Count')
axes[0,1].tick_params(axis='x', rotation=45)

# Average monthly charges by segment
axes[1,0].bar(segment_analysis.index, segment_analysis['AvgMonthlyCharges'])
axes[1,0].set_title('Average Monthly Charges by Segment', fontweight='bold')
axes[1,0].set_ylabel('Monthly Charges ($)')
axes[1,0].tick_params(axis='x', rotation=45)

# Average tenure by segment
axes[1,1].bar(segment_analysis.index, segment_analysis['AvgTenure'])
axes[1,1].set_title('Average Tenure by Segment', fontweight='bold')
axes[1,1].set_ylabel('Tenure (months)')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 7. Statistical Analysis {#statistical-analysis}

Let's perform statistical tests to validate our findings.

In [ ]:
# Chi-square tests for categorical variables
def chi_square_test(feature):
    """Perform chi-square test for independence"""
    contingency_table = pd.crosstab(df_clean[feature], df_clean['Churn'])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    return {
        'feature': feature,
        'chi2': chi2,
        'p_value': p_value,
        'significant': p_value < 0.05
    }

# Test categorical variables
categorical_features = ['gender', 'SeniorCitizenFlag', 'Partner', 'Dependents', 'PhoneService', 
                       'InternetService', 'Contract', 'PaperlessBilling', 'PaymentMethod']

chi_square_results = []
for feature in categorical_features:
    result = chi_square_test(feature)
    chi_square_results.append(result)

chi_square_df = pd.DataFrame(chi_square_results)
chi_square_df = chi_square_df.sort_values('chi2', ascending=False)

print("Chi-Square Test Results (Independence from Churn):")
print("="*60)
print(chi_square_df.to_string(index=False))
print(f"\nSignificant features (p < 0.05): {chi_square_df['significant'].sum()}/{len(chi_square_df)}")

In [ ]:
# T-tests for numerical variables
def t_test_analysis(feature):
    """Perform t-test for numerical features"""
    churned = df_clean[df_clean['Churn'] == 'Yes'][feature]
    retained = df_clean[df_clean['Churn'] == 'No'][feature]
    
    # Perform independent t-test
    t_stat, p_value = stats.ttest_ind(churned, retained)
    
    return {
        'feature': feature,
        'churned_mean': churned.mean(),
        'retained_mean': retained.mean(),
        'mean_difference': churned.mean() - retained.mean(),
        't_statistic': t_stat,
        'p_value': p_value,
        'significant': p_value < 0.05
    }

numerical_features = ['tenure', 'MonthlyCharges', 'TotalCharges', 'AdditionalServicesCount']

t_test_results = []
for feature in numerical_features:
    result = t_test_analysis(feature)
    t_test_results.append(result)

t_test_df = pd.DataFrame(t_test_results)
t_test_df = t_test_df.round(3)

print("\nT-Test Results (Churned vs Retained):")
print("="*80)
print(t_test_df.to_string(index=False))
print(f"\nSignificant differences (p < 0.05): {t_test_df['significant'].sum()}/{len(t_test_df)}")

## 8. Customer Segmentation Analysis {#segmentation}

Let's dive deeper into customer segments and their characteristics.

In [ ]:
# Detailed analysis of high-risk segments
high_risk_threshold = 0.4  # 40% churn rate

# Find high-risk combinations
risk_analysis = []

# Contract + Internet Service
for contract in df_clean['Contract'].unique():
    for internet in df_clean['InternetService'].unique():
        subset = df_clean[(df_clean['Contract'] == contract) & (df_clean['InternetService'] == internet)]
        if len(subset) > 50:  # Only consider segments with enough customers
            churn_rate = (subset['Churn'] == 'Yes').mean()
            if churn_rate > high_risk_threshold:
                risk_analysis.append({
                    'Segment': f'{contract} + {internet}',
                    'Customer_Count': len(subset),
                    'Churn_Rate': churn_rate,
                    'Avg_Monthly_Charges': subset['MonthlyCharges'].mean(),
                    'Avg_Tenure': subset['tenure'].mean()
                })

# Payment Method + Contract
for payment in df_clean['PaymentMethod'].unique():
    for contract in df_clean['Contract'].unique():
        subset = df_clean[(df_clean['PaymentMethod'] == payment) & (df_clean['Contract'] == contract)]
        if len(subset) > 50:
            churn_rate = (subset['Churn'] == 'Yes').mean()
            if churn_rate > high_risk_threshold:
                risk_analysis.append({
                    'Segment': f'{payment} + {contract}',
                    'Customer_Count': len(subset),
                    'Churn_Rate': churn_rate,
                    'Avg_Monthly_Charges': subset['MonthlyCharges'].mean(),
                    'Avg_Tenure': subset['tenure'].mean()
                })

high_risk_df = pd.DataFrame(risk_analysis)
if not high_risk_df.empty:
    high_risk_df = high_risk_df.sort_values('Churn_Rate', ascending=False).round(2)
    high_risk_df = high_risk_df.drop_duplicates(subset=['Segment'])
    
    print("High-Risk Customer Segments (>40% Churn Rate):")
    print("="*80)
    print(high_risk_df.to_string(index=False))
else:
    print("No high-risk segments found with the current threshold.")

In [ ]:
# Service adoption patterns
service_columns = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

# Calculate churn rate by number of additional services
services_churn = df_clean.groupby('AdditionalServicesCount')['Churn'].apply(lambda x: (x=='Yes').mean()).reset_index()
services_count = df_clean.groupby('AdditionalServicesCount').size().reset_index(name='Customer_Count')
services_analysis = services_churn.merge(services_count, on='AdditionalServicesCount')

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Churn rate by number of services
axes[0].plot(services_analysis['AdditionalServicesCount'], services_analysis['Churn'], 
            marker='o', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Additional Services')
axes[0].set_ylabel('Churn Rate')
axes[0].set_title('Churn Rate vs Number of Additional Services', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Customer distribution by number of services
axes[1].bar(services_analysis['AdditionalServicesCount'], services_analysis['Customer_Count'])
axes[1].set_xlabel('Number of Additional Services')
axes[1].set_ylabel('Number of Customers')
axes[1].set_title('Customer Distribution by Additional Services', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nServices Analysis:")
print("="*40)
print(services_analysis.round(3).to_string(index=False))

In [ ]:
# Individual service impact analysis
service_impact = []

for service in service_columns:
    # Calculate churn rate for each service status
    service_churn = df_clean.groupby(service)['Churn'].apply(lambda x: (x=='Yes').mean())
    
    if 'Yes' in service_churn.index and 'No' in service_churn.index:
        impact = service_churn['No'] - service_churn['Yes']  # Positive means service reduces churn
        service_impact.append({
            'Service': service,
            'Churn_Rate_With_Service': service_churn['Yes'],
            'Churn_Rate_Without_Service': service_churn['No'],
            'Impact': impact,
            'Customers_With_Service': (df_clean[service] == 'Yes').sum(),
            'Adoption_Rate': (df_clean[service] == 'Yes').mean()
        })

service_impact_df = pd.DataFrame(service_impact)
service_impact_df = service_impact_df.sort_values('Impact', ascending=False).round(3)

print("\nService Impact on Churn (Positive Impact = Reduces Churn):")
print("="*80)
print(service_impact_df.to_string(index=False))

# Visualize service impact
plt.figure(figsize=(12, 8))
colors = ['green' if x > 0 else 'red' for x in service_impact_df['Impact']]
bars = plt.bar(service_impact_df['Service'], service_impact_df['Impact'], color=colors, alpha=0.7)
plt.axhline(y=0, color='black', linestyle='-', alpha=0.5)
plt.title('Service Impact on Churn Reduction', fontsize=14, fontweight='bold')
plt.xlabel('Service')
plt.ylabel('Churn Rate Difference (Without - With Service)')
plt.xticks(rotation=45)

# Add value labels on bars
for bar, value in zip(bars, service_impact_df['Impact']):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + (0.01 if height > 0 else -0.01),
             f'{value:.3f}', ha='center', va='bottom' if height > 0 else 'top', fontweight='bold')

plt.tight_layout()
plt.show()

## 9. Churn Pattern Analysis {#churn-patterns}

Let's identify specific patterns that lead to churn.

In [ ]:
# Tenure-based churn analysis
# Create more detailed tenure groups
df_clean['DetailedTenureGroup'] = pd.cut(df_clean['tenure'], 
                                       bins=[0, 3, 6, 12, 24, 36, 48, 72], 
                                       labels=['0-3m', '3-6m', '6-12m', '1-2y', '2-3y', '3-4y', '4y+'])

tenure_churn = df_clean.groupby('DetailedTenureGroup').agg({
    'Churn': lambda x: (x=='Yes').mean(),
    'customerID': 'count',
    'MonthlyCharges': 'mean',
    'TotalCharges': 'mean'
}).round(2)

tenure_churn.columns = ['Churn_Rate', 'Customer_Count', 'Avg_Monthly_Charges', 'Avg_Total_Charges']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Churn rate by tenure group
axes[0,0].plot(range(len(tenure_churn)), tenure_churn['Churn_Rate'], marker='o', linewidth=2, markersize=8)
axes[0,0].set_xticks(range(len(tenure_churn)))
axes[0,0].set_xticklabels(tenure_churn.index, rotation=45)
axes[0,0].set_title('Churn Rate by Detailed Tenure Groups', fontweight='bold')
axes[0,0].set_ylabel('Churn Rate')
axes[0,0].grid(True, alpha=0.3)

# Customer count by tenure group
axes[0,1].bar(range(len(tenure_churn)), tenure_churn['Customer_Count'])
axes[0,1].set_xticks(range(len(tenure_churn)))
axes[0,1].set_xticklabels(tenure_churn.index, rotation=45)
axes[0,1].set_title('Customer Distribution by Tenure Groups', fontweight='bold')
axes[0,1].set_ylabel('Customer Count')

# Average monthly charges by tenure
axes[1,0].bar(range(len(tenure_churn)), tenure_churn['Avg_Monthly_Charges'])
axes[1,0].set_xticks(range(len(tenure_churn)))
axes[1,0].set_xticklabels(tenure_churn.index, rotation=45)
axes[1,0].set_title('Average Monthly Charges by Tenure Groups', fontweight='bold')
axes[1,0].set_ylabel('Monthly Charges ($)')

# Average total charges by tenure
axes[1,1].bar(range(len(tenure_churn)), tenure_churn['Avg_Total_Charges'])
axes[1,1].set_xticks(range(len(tenure_churn)))
axes[1,1].set_xticklabels(tenure_churn.index, rotation=45)
axes[1,1].set_title('Average Total Charges by Tenure Groups', fontweight='bold')
axes[1,1].set_ylabel('Total Charges ($)')

plt.tight_layout()
plt.show()

print("\nDetailed Tenure Analysis:")
print("="*60)
print(tenure_churn)

In [ ]:
# Early churn indicators (customers with tenure < 12 months)
early_customers = df_clean[df_clean['tenure'] <= 12].copy()
print(f"Early customers analysis (tenure ≤ 12 months): {len(early_customers):,} customers")
print(f"Early churn rate: {(early_customers['Churn'] == 'Yes').mean():.2%}")

# Top factors for early churn
early_churn_factors = []

categorical_features = ['gender', 'SeniorCitizenFlag', 'Partner', 'Dependents', 'InternetService', 
                       'Contract', 'PaymentMethod', 'PaperlessBilling']

for feature in categorical_features:
    feature_churn = early_customers.groupby(feature)['Churn'].apply(lambda x: (x=='Yes').mean())
    max_churn = feature_churn.max()
    max_category = feature_churn.idxmax()
    
    early_churn_factors.append({
        'Feature': feature,
        'Highest_Risk_Category': max_category,
        'Churn_Rate': max_churn,
        'Customer_Count': (early_customers[feature] == max_category).sum()
    })

early_churn_df = pd.DataFrame(early_churn_factors)
early_churn_df = early_churn_df.sort_values('Churn_Rate', ascending=False)

print("\nEarly Churn Risk Factors (≤ 12 months tenure):")
print("="*70)
print(early_churn_df.round(3).to_string(index=False))

In [ ]:
# Customer lifetime value analysis
plt.figure(figsize=(16, 10))

# Create subplots
gs = plt.GridSpec(2, 3, figsize=(18, 12))

# CLV distribution by churn
ax1 = plt.subplot(gs[0, :])
churned = df_clean[df_clean['Churn'] == 'Yes']['TotalCharges']
retained = df_clean[df_clean['Churn'] == 'No']['TotalCharges']

ax1.hist(retained, bins=50, alpha=0.7, label='Retained', color='blue', density=True)
ax1.hist(churned, bins=50, alpha=0.7, label='Churned', color='red', density=True)
ax1.axvline(retained.mean(), color='blue', linestyle='--', alpha=0.8, label=f'Retained Mean: ${retained.mean():.0f}')
ax1.axvline(churned.mean(), color='red', linestyle='--', alpha=0.8, label=f'Churned Mean: ${churned.mean():.0f}')
ax1.set_title('Total Charges Distribution by Churn Status', fontsize=14, fontweight='bold')
ax1.set_xlabel('Total Charges ($)')
ax1.set_ylabel('Density')
ax1.legend()

# CLV tier analysis
ax2 = plt.subplot(gs[1, 0])
clv_churn = df_clean.groupby('CLV_Tier')['Churn'].apply(lambda x: (x=='Yes').mean())
ax2.bar(clv_churn.index, clv_churn.values)
ax2.set_title('Churn Rate by CLV Tier', fontweight='bold')
ax2.set_ylabel('Churn Rate')
ax2.tick_params(axis='x', rotation=45)

# Monthly charges tier analysis
ax3 = plt.subplot(gs[1, 1])
monthly_churn = df_clean.groupby('MonthlyChargesTier')['Churn'].apply(lambda x: (x=='Yes').mean())
ax3.bar(monthly_churn.index, monthly_churn.values)
ax3.set_title('Churn Rate by Monthly Charges Tier', fontweight='bold')
ax3.set_ylabel('Churn Rate')
ax3.tick_params(axis='x', rotation=45)

# Family status analysis
ax4 = plt.subplot(gs[1, 2])
family_churn = df_clean.groupby('HasFamily')['Churn'].apply(lambda x: (x=='Yes').mean())
ax4.bar(family_churn.index, family_churn.values)
ax4.set_title('Churn Rate by Family Status', fontweight='bold')
ax4.set_ylabel('Churn Rate')

plt.tight_layout()
plt.show()

# Summary statistics
print("\nCustomer Lifetime Value Impact:")
print("="*50)
print(f"Average CLV - Retained: ${retained.mean():.2f}")
print(f"Average CLV - Churned: ${churned.mean():.2f}")
print(f"Revenue Impact per Churned Customer: ${retained.mean() - churned.mean():.2f}")
print(f"Total Revenue at Risk: ${(retained.mean() - churned.mean()) * len(churned):,.2f}")

## 10. Key Insights and Business Recommendations {#insights}

Based on our comprehensive analysis, here are the key findings and actionable recommendations.

In [ ]:
# Summary of key metrics
print("🔍 KEY BUSINESS INSIGHTS & RECOMMENDATIONS")
print("="*80)

# Overall metrics
total_customers = len(df_clean)
churned_customers = (df_clean['Churn'] == 'Yes').sum()
churn_rate = churned_customers / total_customers
avg_revenue_per_customer = df_clean['TotalCharges'].mean()
revenue_at_risk = churned_customers * avg_revenue_per_customer

print(f"\n📊 BUSINESS OVERVIEW:")
print(f"   • Total Customers: {total_customers:,}")
print(f"   • Churned Customers: {churned_customers:,}")
print(f"   • Overall Churn Rate: {churn_rate:.1%}")
print(f"   • Average Customer Lifetime Value: ${avg_revenue_per_customer:.2f}")
print(f"   • Annual Revenue at Risk: ${revenue_at_risk:,.2f}")

print(f"\n🚨 HIGHEST RISK SEGMENTS:")
# Top risk factors
risk_factors = [
    ('Month-to-month contracts', df_clean[df_clean['Contract'] == 'Month-to-month']['Churn'].apply(lambda x: x=='Yes').mean()),
    ('Electronic check payments', df_clean[df_clean['PaymentMethod'] == 'Electronic check']['Churn'].apply(lambda x: x=='Yes').mean()),
    ('Fiber optic internet', df_clean[df_clean['InternetService'] == 'Fiber optic']['Churn'].apply(lambda x: x=='Yes').mean()),
    ('Paperless billing', df_clean[df_clean['PaperlessBilling'] == 'Yes']['Churn'].apply(lambda x: x=='Yes').mean()),
    ('New customers (<12 months)', df_clean[df_clean['tenure'] < 12]['Churn'].apply(lambda x: x=='Yes').mean())
]

for factor, rate in sorted(risk_factors, key=lambda x: x[1], reverse=True):
    print(f"   • {factor}: {rate:.1%} churn rate")

print(f"\n💡 STRATEGIC RECOMMENDATIONS:")
print(f"   
   1. CONTRACT OPTIMIZATION:
      • Incentivize longer-term contracts with discounts
      • Offer flexible upgrade paths for month-to-month customers
      • Implement early warning system for contract renewals
   
   2. PAYMENT METHOD STRATEGY:
      • Promote automatic payment methods with incentives
      • Reduce friction in electronic check payments
      • Offer payment method migration bonuses
   
   3. NEW CUSTOMER ONBOARDING:
      • Enhance first 90-day customer experience
      • Implement proactive customer success program
      • Provide dedicated support for new customers
   
   4. SERVICE BUNDLING:
      • Promote security and support services (reduce churn)
      • Create value-added service packages
      • Educate customers on service benefits
   
   5. PRICING STRATEGY:
      • Review fiber optic pricing and value proposition
      • Implement retention offers for high-risk segments
      • Consider loyalty discounts for long-term customers
   ")

print(f"\n📈 EXPECTED IMPACT:")
# Potential impact calculations
potential_retention = churned_customers * 0.25  # 25% improvement
revenue_recovery = potential_retention * avg_revenue_per_customer

print(f"   • Target: 25% reduction in churn rate")
print(f"   • Customers retained: {potential_retention:.0f}")
print(f"   • Revenue recovery: ${revenue_recovery:,.2f}")
print(f"   • ROI: Implementing retention programs could recover significant revenue")

In [ ]:
# Create a final summary dashboard
fig = plt.figure(figsize=(20, 16))

# Create a complex grid layout
gs = plt.GridSpec(4, 4, height_ratios=[1, 1, 1, 1], width_ratios=[1, 1, 1, 1])

# 1. Churn rate by contract type
ax1 = fig.add_subplot(gs[0, 0])
contract_churn = df_clean.groupby('Contract')['Churn'].apply(lambda x: (x=='Yes').mean())
bars1 = ax1.bar(contract_churn.index, contract_churn.values, color=['red', 'orange', 'green'])
ax1.set_title('Churn by Contract Type', fontweight='bold', fontsize=10)
ax1.tick_params(axis='x', rotation=45, labelsize=8)
for i, v in enumerate(contract_churn.values):
    ax1.text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold', fontsize=8)

# 2. Churn rate by payment method
ax2 = fig.add_subplot(gs[0, 1])
payment_churn = df_clean.groupby('PaymentMethod')['Churn'].apply(lambda x: (x=='Yes').mean())
bars2 = ax2.barh(payment_churn.index, payment_churn.values, color=['red', 'orange', 'yellow', 'green'])
ax2.set_title('Churn by Payment Method', fontweight='bold', fontsize=10)
ax2.tick_params(axis='y', labelsize=8)
for i, v in enumerate(payment_churn.values):
    ax2.text(v + 0.02, i, f'{v:.1%}', va='center', fontweight='bold', fontsize=8)

# 3. Tenure distribution
ax3 = fig.add_subplot(gs[0, 2])
churned_tenure = df_clean[df_clean['Churn'] == 'Yes']['tenure']
retained_tenure = df_clean[df_clean['Churn'] == 'No']['tenure']
ax3.hist([retained_tenure, churned_tenure], bins=20, alpha=0.7, label=['Retained', 'Churned'], color=['blue', 'red'])
ax3.set_title('Tenure Distribution', fontweight='bold', fontsize=10)
ax3.set_xlabel('Tenure (months)', fontsize=8)
ax3.legend(fontsize=8)

# 4. Monthly charges distribution
ax4 = fig.add_subplot(gs[0, 3])
churned_charges = df_clean[df_clean['Churn'] == 'Yes']['MonthlyCharges']
retained_charges = df_clean[df_clean['Churn'] == 'No']['MonthlyCharges']
ax4.hist([retained_charges, churned_charges], bins=20, alpha=0.7, label=['Retained', 'Churned'], color=['blue', 'red'])
ax4.set_title('Monthly Charges Distribution', fontweight='bold', fontsize=10)
ax4.set_xlabel('Monthly Charges ($)', fontsize=8)
ax4.legend(fontsize=8)

# 5. Internet service impact
ax5 = fig.add_subplot(gs[1, 0])
internet_churn = df_clean.groupby('InternetService')['Churn'].apply(lambda x: (x=='Yes').mean())
bars5 = ax5.bar(internet_churn.index, internet_churn.values, color=['green', 'red', 'orange'])
ax5.set_title('Churn by Internet Service', fontweight='bold', fontsize=10)
ax5.tick_params(axis='x', rotation=45, labelsize=8)
for i, v in enumerate(internet_churn.values):
    ax5.text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold', fontsize=8)

# 6. Service adoption impact
ax6 = fig.add_subplot(gs[1, 1])
service_churn = df_clean.groupby('AdditionalServicesCount')['Churn'].apply(lambda x: (x=='Yes').mean())
ax6.plot(service_churn.index, service_churn.values, marker='o', linewidth=2, markersize=6)
ax6.set_title('Churn by # Additional Services', fontweight='bold', fontsize=10)
ax6.set_xlabel('Number of Services', fontsize=8)
ax6.set_ylabel('Churn Rate', fontsize=8)
ax6.grid(True, alpha=0.3)

# 7. Family status impact
ax7 = fig.add_subplot(gs[1, 2])
family_churn = df_clean.groupby('HasFamily')['Churn'].apply(lambda x: (x=='Yes').mean())
bars7 = ax7.bar(family_churn.index, family_churn.values, color=['red', 'green'])
ax7.set_title('Churn by Family Status', fontweight='bold', fontsize=10)
for i, v in enumerate(family_churn.values):
    ax7.text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold', fontsize=8)

# 8. Senior citizen impact
ax8 = fig.add_subplot(gs[1, 3])
senior_churn = df_clean.groupby('SeniorCitizenFlag')['Churn'].apply(lambda x: (x=='Yes').mean())
bars8 = ax8.bar(senior_churn.index, senior_churn.values, color=['green', 'orange'])
ax8.set_title('Churn by Senior Citizen', fontweight='bold', fontsize=10)
for i, v in enumerate(senior_churn.values):
    ax8.text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold', fontsize=8)

# 9-12. Service-specific analysis
services = ['OnlineSecurity', 'TechSupport', 'StreamingTV', 'StreamingMovies']
for i, service in enumerate(services):
    ax = fig.add_subplot(gs[2, i])
    service_data = df_clean.groupby(service)['Churn'].apply(lambda x: (x=='Yes').mean())
    bars = ax.bar(service_data.index, service_data.values)
    ax.set_title(f'{service} Impact', fontweight='bold', fontsize=10)
    ax.tick_params(axis='x', rotation=45, labelsize=8)
    for j, v in enumerate(service_data.values):
        ax.text(j, v + 0.01, f'{v:.1%}', ha='center', fontweight='bold', fontsize=8)

# 13. Customer segment analysis
ax13 = fig.add_subplot(gs[3, :])
segment_churn = df_clean.groupby('CustomerSegment')['Churn'].apply(lambda x: (x=='Yes').mean()).sort_values(ascending=False)
bars13 = ax13.bar(range(len(segment_churn)), segment_churn.values, 
                 color=['darkred', 'red', 'orange', 'yellow', 'lightgreen', 'green'])
ax13.set_xticks(range(len(segment_churn)))
ax13.set_xticklabels(segment_churn.index, rotation=45, ha='right')
ax13.set_title('Churn Rate by Customer Segment', fontweight='bold', fontsize=12)
ax13.set_ylabel('Churn Rate')

# Add percentage labels
for i, v in enumerate(segment_churn.values):
    ax13.text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold', fontsize=10)

plt.suptitle('Telecom Customer Churn Analysis - Executive Dashboard', 
             fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.subplots_adjust(top=0.95)
plt.show()

print("\n📋 DASHBOARD SUMMARY:")
print("   This executive dashboard provides a comprehensive view of churn patterns")
print("   across all key customer segments and service categories.")
print("   Use this for strategic decision-making and resource allocation.")

## 11. Conclusion {#conclusion}

### Summary of Key Findings

This comprehensive EDA has revealed several critical insights about customer churn in the telecom industry:

#### 🔍 **Primary Churn Drivers:**
1. **Contract Type**: Month-to-month contracts show significantly higher churn rates
2. **Payment Method**: Electronic check users are at highest risk
3. **Tenure**: New customers (especially first 12 months) are most vulnerable
4. **Service Type**: Fiber optic internet customers have higher churn rates
5. **Customer Demographics**: Senior citizens show elevated churn risk

#### 📊 **Business Impact:**
- Overall churn rate affects a significant portion of the customer base
- Early-tenure customers represent the highest risk segment
- Service bundling shows potential for reducing churn
- Customer lifetime value varies significantly between churned and retained customers

#### 💡 **Strategic Opportunities:**
1. **Retention Programs**: Focus on high-risk segments identified in analysis
2. **Product Strategy**: Enhance value proposition for vulnerable service categories
3. **Customer Experience**: Improve onboarding and early-tenure support
4. **Pricing Optimization**: Develop targeted offers for at-risk segments

### Next Steps
1. **Predictive Modeling**: Use insights to build churn prediction models
2. **A/B Testing**: Test retention strategies on identified segments
3. **Customer Journey Mapping**: Deep-dive into high-risk customer experiences
4. **Performance Monitoring**: Track KPIs and intervention effectiveness

### Data Quality Notes
- Dataset appears complete with minimal missing values
- All key business metrics are represented
- Statistical tests confirm significance of major findings
- Recommendations are data-driven and actionable

---

*This analysis provides a solid foundation for developing targeted retention strategies and improving overall customer satisfaction in the telecom business.*